In [ ]:
%pip install wikipedia wikipedia-api
%pip install langchain langchain-community langchain-huggingface
%pip install faiss-cpu faiss-gpu
%pip install transformers
%pip install torch
%pip install bitsandbytes
%pip install accelerate

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=4e067ccf0096cd4600caa89d8c3374388b972a366153c31704da56a89ab3f082
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14347 sha256=02248b13f1775dd43ada1db4706568dd3e27035a071abf6fddceb30629744997
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia wikipedia-api
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
import wikipedia

titles = wikipedia.search("Polar bear, weight", results=7)
pages = []
for t in titles:
    try:
        pages.append(wikipedia.page(t, auto_suggest=False))
    except Exception as e:
        warnings.warn(str(e))

pages

[<WikipediaPage 'Polar bear'>,
 <WikipediaPage 'Polar bear conservation'>,
 <WikipediaPage 'Bear'>,
 <WikipediaPage 'American black bear'>,
 <WikipediaPage 'Knut (polar bear)'>,
 <WikipediaPage 'Siku (polar bear)'>,
 <WikipediaPage 'Brown bear'>]

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

def split_text(pages, chunk_size=1000, chunk_overlap=100):
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for page in pages:
        chunks.extend(splitter.split_text(page.content))
    return chunks

text_chunks = split_text(pages)


In [ ]:
text_chunks[190]

"== References ==\n\n\n== Notes ==\n\n\n== Bibliography ==\nVaisfeld, M.A. and Chestin I. E., ed. (1993). Bears: Brown Bear, Polar Bear, Asian Black Bear. Distribution, ecology, use and protection (in Russian and English). Moscow: Nauka. ISBN 978-5020035676.\n\n\n== Further reading ==\nMonaghan, Nigel T. 2023 The Brown Bear (Ursus arctos L) in Ireland. Irish Naturalist' Journal. 1 - 19\n\n\n== External links ==\n\nBrown Bear profile from National GeographicFurth\nBear Hunting Altered Genetics More Than Ice Age Isolation\nAncient Fossil Offers New Clues To Brown Bears Past"

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                        encode_kwargs={'normalize_embeddings': True})

faiss_index = FAISS.from_texts(text_chunks, embedding_model)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
faiss_index.similarity_search("What is this species weight? Keywords : Polar bear, weight", k=2)[0].page_content

In [ ]:
def retrieve_relevant_texts(query, faiss_index, top_k=5):
    return faiss_index.similarity_search(query, k=top_k)

user_query = "What is this species weight?"
keywords = "Polar bear, weight"
relevant_texts = retrieve_relevant_texts(user_query+"\nKeywords :"+keywords, faiss_index, 2)
relevant_texts = [text.page_content for text in relevant_texts]
for text in relevant_texts:
    print(text)


The polar bear is the largest living species of bear and land carnivore, though some brown bear subspecies like the Kodiak bear can rival it in size. Males are generally 200–250 cm (6.6–8.2 ft) long with a weight of 300–800 kg (660–1,760 lb). Females are smaller at 180–200 cm (5.9–6.6 ft) with a weight of 150–300 kg (330–660 lb). Sexual dimorphism in the species is particularly high compared with most other mammals. Male polar bears also have proportionally larger heads than females. The weight of polar bears fluctuates during the year, as they can bulk up on fat and increase their mass by 50 percent. A fattened, pregnant female can weigh as much as 500 kg (1,100 lb). Adults may stand 130–160 cm (4.3–5.2 ft) tall at the shoulder. The tail is 76–126 mm (3.0–5.0 in) long. The largest polar bear on record, reportedly weighing 1,002 kg (2,209 lb), was a male shot at Kotzebue Sound in northwestern Alaska in 1960.
Compared with the brown bear, this species has a more slender build, with a na

In [ ]:
len(relevant_texts)

2

In [ ]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
import requests
from PIL import Image
import re

device = "cuda" if torch.cuda.is_available() else "cpu"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto",
)

def generate_answer_from_retrieved(retrieved_texts, query, image):
    context = "\n\n".join(retrieved_texts)
    input_text = f"Question: {query}\nYou can use the following information from Wikipedia to provide your answer\n\nContext: {context}"

    conversation = [
        {

          "role": "user",
          "content": [
              {"type": "text", "text": input_text},
              {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs = processor(images = image, text = prompt, return_tensors="pt").to(device)

    outputs = model.generate(**inputs, max_new_tokens=250)
    full_output = processor.decode(outputs[0], skip_special_tokens=True)
    prompt_length = len(processor.decode(inputs["input_ids"][0], skip_special_tokens=True))

    answer = full_output[prompt_length:].strip()
    return answer

image = Image.open(requests.get("https://upload.wikimedia.org/wikipedia/commons/6/66/Polar_Bear_-_Alaska_%28cropped%29.jpg", stream=True).raw)
final_answer = generate_answer_from_retrieved(relevant_texts, user_query, image)
print(final_answer)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The polar bear is the largest living species of bear and land carnivore. Adult males typically weigh between 350-700 kg (770-1,540 lb) and measure 2.4-3 m (7 ft 10 in - 9 ft 10 in) in total length. Females are smaller, weighing between 150-300 kg (330-660 lb) and measuring 1.8-2.4 m (5 ft 11 in - 7 ft 10 in) in length. The polar bear has a slender build with a narrower, flatter and smaller skull, a longer neck, and a lower shoulder hump. It has 34-42 teeth including 12 incisors, 4 canines, 8-16 premolars and 10 molars. The bear has large paws, with the front paws being broader than the back. The feet are hairier than in other bear species, providing warmth and friction when stepping on snow and sea ice. The cla


In [ ]:
answer = generate_answer_from_retrieved(relevant_texts, user_query+"Only one sentence please.", image)
print(answer)

The polar bear is the largest living species of bear and land carnivore, with adult males weighing 350-700 kg and measuring 2.4-3 m in length. They have a slender build, a narrower, flatter and smaller skull, a longer neck, and a lower shoulder hump. The snout profile is curved, resembling a "Roman nose". They have 34-42 teeth including 12 incisors, 4 canines, 8-16 premolars and 10 molars. The feet are hairier than in other bear species, providing warmth and friction when stepping on snow and sea ice. The claws are small but sharp and hooked and are used both to snatch prey and climb onto ice.


In [ ]:
# Base-model sorgu örneği
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": user_query},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
inputs = processor(images = image, text = prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=250)
full_output = processor.decode(outputs[0], skip_special_tokens=True)
prompt_length = len(processor.decode(inputs["input_ids"][0], skip_special_tokens=True))
print(full_output[prompt_length:].strip())

The polar bear in the image is a large, adult male. It is not possible to determine its exact weight from the image, but it is evident that it is a large and heavy animal. Polar bears are known for their impressive size and strength, and they can weigh anywhere from 900 to 1,600 pounds (410 to 725 kilograms) depending on factors such as age, sex, and body condition.
